In [ ]:
# 📋 곱창집 입지 분석 최종 보고서
# 06_final_report.ipynb
#
# 목적: 전체 분석 결과를 종합하여 경영진용 요약 + 창업자용 실행 가이드 작성
# 기능: Executive Summary, 실행 로드맵, 체크리스트, 투자 계획서
# ================================================================================

# 📚 라이브러리 임포트
# ================================================================
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("📋 곱창집 입지 분석 최종 보고서 작성 시작!")
print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)

# 📂 경로 설정 및 데이터 로드
# ================================================================
OUTPUT_PATH = Path("../docs")

# 전체 분석 결과 파일들 로드
try:
    # TOP 30 결과
    top30_df = pd.read_csv(OUTPUT_PATH / "gopchang_top30_locations_v2.csv", encoding='utf-8-sig')
    
    # 좌표 정보
    coords_df = pd.read_csv(OUTPUT_PATH / "gopchang_top30_coordinates.csv", encoding='utf-8-sig')
    
    # 전체 분석 데이터
    full_analysis = pd.read_csv(OUTPUT_PATH / "gopchang_kpi_analysis_v2.csv", encoding='utf-8-sig')
    
    print("✅ 모든 분석 결과 파일 로드 완료")
    print(f"   📊 TOP 30: {top30_df.shape}")
    print(f"   🗺️ 좌표: {coords_df.shape}")
    print(f"   📈 전체: {full_analysis.shape}")
    
except Exception as e:
    print(f"❌ 파일 로드 오류: {e}")
    print("💡 먼저 이전 단계들을 완료해주세요!")

print("=" * 70)

# 📊 Executive Summary 데이터 준비
# ================================================================
print("\n📊 Executive Summary 데이터 준비")
print("-" * 25)

# 핵심 지표 계산
total_districts = len(full_analysis)
analyzed_districts = len(top30_df)
top10_threshold = top30_df.iloc[9]['곱창집_적합도_v2'] if len(top30_df) >= 10 else 0

# 상권 등급 분류
excellent_districts = (top30_df['곱창집_적합도_v2'] >= 40).sum()
good_districts = ((top30_df['곱창집_적합도_v2'] >= 35) & (top30_df['곱창집_적합도_v2'] < 40)).sum()
fair_districts = ((top30_df['곱창집_적합도_v2'] >= 30) & (top30_df['곱창집_적합도_v2'] < 35)).sum()

# 투자 규모별 분류 (야간유동인구 + 매출 기준)
high_investment = top30_df[
    (top30_df['T1_야간유동인구'] >= 1500000) | 
    (top30_df['한식_월매출'] >= 50000000000)
]
medium_investment = top30_df[
    ((top30_df['T1_야간유동인구'] >= 800000) & (top30_df['T1_야간유동인구'] < 1500000)) |
    ((top30_df['한식_월매출'] >= 20000000000) & (top30_df['한식_월매출'] < 50000000000))
]
low_investment = top30_df[
    (top30_df['T1_야간유동인구'] < 800000) & 
    (top30_df['한식_월매출'] < 20000000000)
]

print(f"📈 핵심 통계:")
print(f"   • 전체 분석 상권: {total_districts:,}개")
print(f"   • 추천 상권 (TOP 30): {analyzed_districts}개")
print(f"   • 최우수 상권 (40점+): {excellent_districts}개")
print(f"   • 우수 상권 (35-40점): {good_districts}개")
print(f"   • 고투자형 상권: {len(high_investment)}개")
print(f"   • 중투자형 상권: {len(medium_investment)}개")
print(f"   • 저투자형 상권: {len(low_investment)}개")

print("=" * 70)

# 🏢 상권 유형별 종합 분석
# ================================================================
print("\n🏢 상권 유형별 종합 분석")
print("-" * 20)

def classify_district_type(name):
    """상권명 기반 유형 분류"""
    if any(keyword in name for keyword in ['강남', '역삼', '선릉', '신논현']):
        return "프리미엄 비즈니스"
    elif any(keyword in name for keyword in ['홍대', '연남', '망리단']):
        return "젊은층 문화"
    elif any(keyword in name for keyword in ['신촌', '대학로', '신림', '건대']):
        return "대학가"
    elif any(keyword in name for keyword in ['명동', '종로', '북창동', '관광특구']):
        return "관광/전통상권"
    elif any(keyword in name for keyword in ['노원', '수유', '불광', '영등포', '노량진']):
        return "지역 거점"
    else:
        return "복합상권"

# TOP 30 상권 유형별 분류
top30_df['상권유형'] = top30_df['상권_코드_명'].apply(classify_district_type)
type_analysis = top30_df.groupby('상권유형').agg({
    '곱창집_적합도_v2': ['count', 'mean', 'max'],
    'T1_야간유동인구': 'mean',
    '한식_월매출': 'mean'
}).round(1)

type_analysis.columns = ['상권수', '평균점수', '최고점수', '평균야간유동', '평균한식매출']

print("📋 상권 유형별 특성:")
for idx, row in type_analysis.iterrows():
    print(f"🏢 {idx}: {row['상권수']}개 상권")
    print(f"   📊 평균 적합도: {row['평균점수']:.1f}점 (최고 {row['최고점수']:.1f}점)")
    print(f"   👥 평균 야간유동: {row['평균야간유동']:,.0f}명")
    print(f"   💰 평균 한식매출: {row['평균한식매출']/100000000:.1f}억원")
    print()

print("=" * 70)

# 💡 핵심 인사이트 및 추천사항
# ================================================================
print("\n💡 핵심 인사이트 및 추천사항")
print("-" * 25)

# 최고 성과 상권 분석
best_district = top30_df.iloc[0]
best_type = classify_district_type(best_district['상권_코드_명'])

# 가성비 최고 상권 (점수 대비 투자비용)
top30_df['투자효율성'] = top30_df['곱창집_적합도_v2'] / (
    (top30_df['한식_월매출'] / 1000000000) + 1
)  # 매출 규모로 투자비용 추정
best_value = top30_df.loc[top30_df['투자효율성'].idxmax()]

# 안정성 최고 상권 (고른 점수 분포)
top30_df['점수편차'] = (
    top30_df[['T1_점수_v2', 'T2_점수_v2', 'C1_점수_v2', 'C2_점수_v2', 'E1_점수_v2']].std(axis=1)
)
most_stable = top30_df.loc[top30_df['점수편차'].idxmin()]

insights = {
    'best_overall': {
        'name': best_district['상권_코드_명'],
        'score': best_district['곱창집_적합도_v2'],
        'type': best_type,
        'reason': '최고 종합 점수'
    },
    'best_value': {
        'name': best_value['상권_코드_명'],
        'score': best_value['곱창집_적합도_v2'],
        'efficiency': best_value['투자효율성'],
        'reason': '최고 투자 효율성'
    },
    'most_stable': {
        'name': most_stable['상권_코드_명'],
        'score': most_stable['곱창집_적합도_v2'],
        'stability': most_stable['점수편차'],
        'reason': '최고 안정성 (균형잡힌 지표)'
    }
}

print("🏆 핵심 추천 상권:")
print(f"1. 최고 종합 성과: {insights['best_overall']['name']}")
print(f"   📊 점수: {insights['best_overall']['score']:.1f}점")
print(f"   🏢 유형: {insights['best_overall']['type']}")
print(f"   💡 이유: 야간유동인구와 상권활성도 최고")

print(f"\n2. 최고 투자 효율: {insights['best_value']['name']}")
print(f"   📊 점수: {insights['best_value']['score']:.1f}점")
print(f"   💰 효율성: {insights['best_value']['efficiency']:.2f}")
print(f"   💡 이유: 적정 투자비용 대비 높은 성과 기대")

print(f"\n3. 최고 안정성: {insights['most_stable']['name']}")
print(f"   📊 점수: {insights['most_stable']['score']:.1f}점")
print(f"   📈 안정성: {insights['most_stable']['stability']:.1f} (낮을수록 좋음)")
print(f"   💡 이유: 모든 지표가 균형잡힌 안전한 선택")

print("=" * 70)

# 📈 시장 규모 및 기회 분석
# ================================================================
print("\n📈 시장 규모 및 기회 분석")
print("-" * 20)

# 야간 시간대 시장 규모 추정
total_night_population = top30_df['T1_야간유동인구'].sum()
total_korean_sales = top30_df['한식_월매출'].sum()

# 곱창집 시장 규모 추정 (한식음식점의 15% 가정)
estimated_gopchang_market = total_korean_sales * 0.15
monthly_market_size = estimated_gopchang_market / 1000000000  # 억원 단위

# 1인당 평균 소비액 추정
avg_consumption_per_person = total_korean_sales / total_night_population

print(f"📊 TOP 30 상권 시장 규모:")
print(f"   👥 총 야간 유동인구: {total_night_population:,}명")
print(f"   💰 총 한식 매출: {total_korean_sales/1000000000:,.0f}억원/월")
print(f"   🍖 곱창집 시장 추정: {monthly_market_size:,.0f}억원/월 (한식의 15%)")
print(f"   💵 1인당 평균 소비: {avg_consumption_per_person:,.0f}원")

# 곱창집 포화도 분석
current_gopchang_density = top30_df['한식_점포수'].mean()
optimal_density = 3  # 상권당 최적 곱창집 수 (경험치)

print(f"\n🏪 곱창집 포화도 분석:")
print(f"   📊 현재 한식점 평균: {current_gopchang_density:.1f}개/상권")
print(f"   🎯 최적 곱창집 수: {optimal_density}개/상권")
print(f"   📈 시장 기회: {'포화' if current_gopchang_density > optimal_density else '여유'}")

print("=" * 70)

# 🎯 상권별 창업 전략 매트릭스
# ================================================================
print("\n🎯 상권별 창업 전략 매트릭스")
print("-" * 25)

def create_strategy_matrix():
    """투자규모 vs 리스크 매트릭스 생성"""
    
    strategy_matrix = []
    
    for idx, row in top30_df.head(15).iterrows():  # TOP 15만 분석
        name = row['상권_코드_명']
        score = row['곱창집_적합도_v2']
        night_pop = row['T1_야간유동인구']
        korean_sales = row['한식_월매출']
        
        # 투자 규모 계산 (매출 규모 기반)
        if korean_sales >= 50000000000:  # 500억 이상
            investment_level = "고투자"
            initial_cost = "8천만원-1.5억원"
        elif korean_sales >= 20000000000:  # 200억 이상
            investment_level = "중투자" 
            initial_cost = "5천만원-8천만원"
        else:
            investment_level = "저투자"
            initial_cost = "3천만원-5천만원"
        
        # 리스크 레벨 (점수 편차 + 경쟁도)
        score_std = row[['T1_점수_v2', 'T2_점수_v2', 'C1_점수_v2', 'C2_점수_v2', 'E1_점수_v2']].std()
        if score_std <= 20:
            risk_level = "저리스크"
        elif score_std <= 30:
            risk_level = "중리스크"
        else:
            risk_level = "고리스크"
        
        # 예상 ROI (점수 기반)
        if score >= 40:
            expected_roi = "20-30%"
            payback_period = "2-3년"
        elif score >= 35:
            expected_roi = "15-25%"
            payback_period = "3-4년"
        elif score >= 30:
            expected_roi = "10-20%"
            payback_period = "4-5년"
        else:
            expected_roi = "5-15%"
            payback_period = "5-6년"
        
        # 추천 타겟
        district_type = classify_district_type(name)
        if district_type == "프리미엄 비즈니스":
            target = "기업 임원, 고소득층"
        elif district_type == "젊은층 문화":
            target = "20-30대, 연인/친구"
        elif district_type == "대학가":
            target = "대학생, 젊은 직장인"
        elif district_type == "관광/전통상권":
            target = "관광객, 외국인"
        else:
            target = "지역주민, 직장인"
        
        strategy_matrix.append({
            'rank': idx + 1,
            'name': name,
            'score': score,
            'investment_level': investment_level,
            'initial_cost': initial_cost,
            'risk_level': risk_level,
            'expected_roi': expected_roi,
            'payback_period': payback_period,
            'target': target,
            'district_type': district_type
        })
    
    return strategy_matrix

strategy_data = create_strategy_matrix()

print("🎯 TOP 15 상권 창업 전략 매트릭스:")
print("=" * 80)
print(f"{'순위':<3} {'상권명':<25} {'투자':<6} {'리스크':<8} {'ROI':<8} {'회수기간':<8}")
print("-" * 80)

for data in strategy_data:
    print(f"{data['rank']:<3} {data['name'][:25]:<25} {data['investment_level']:<6} "
          f"{data['risk_level']:<8} {data['expected_roi']:<8} {data['payback_period']:<8}")

print("=" * 70)

# 📋 창업자용 실행 체크리스트
# ================================================================
print("\n📋 창업자용 실행 체크리스트")
print("-" * 20)

checklist_items = {
    "사전 준비 단계 (창업 3-6개월 전)": [
        "목표 상권 3-5곳 선정 및 현장 답사",
        "상권별 임대료 조사 (권리금, 보증금, 월세)",
        "경쟁업체 분석 (메뉴, 가격, 인테리어, 고객층)",
        "초기 투자비용 계획 (인테리어, 장비, 운전자금)",
        "사업계획서 작성 및 자금 조달",
        "음식점 창업 관련 교육 이수",
        "식품위생교육 이수 및 영업신고 준비"
    ],
    "입지 선정 단계 (창업 2-3개월 전)": [
        "최종 상권 결정 (TOP 3 중 선택)",
        "점포 계약 조건 협상 (임대료, 계약기간)",
        "상권 특성에 맞는 컨셉 확정",
        "타겟 고객층 구체화",
        "메뉴 구성 및 가격 정책 수립",
        "인테리어 디자인 및 시공업체 선정",
        "주방 장비 및 설비 계획"
    ],
    "개업 준비 단계 (창업 1-2개월 전)": [
        "인테리어 공사 및 장비 설치",
        "직원 채용 및 교육",
        "식자재 공급업체 선정",
        "POS 시스템 및 결제 시스템 구축",
        "마케팅 계획 수립 (SNS, 전단지, 이벤트)",
        "메뉴 테스트 및 최종 조정",
        "위생 점검 및 영업신고"
    ],
    "개업 초기 단계 (개업 후 1-3개월)": [
        "소프트 오픈 (지인, 가족 대상)",
        "그랜드 오픈 이벤트 진행",
        "고객 피드백 수집 및 개선",
        "일일 매출 및 원가 관리",
        "직원 재교육 및 서비스 개선",
        "단골 고객 확보 전략 실행",
        "온라인 리뷰 관리 및 홍보"
    ]
}

for stage, items in checklist_items.items():
    print(f"\n📅 {stage}:")
    for i, item in enumerate(items, 1):
        print(f"   {i}. ☐ {item}")

print("=" * 70)

# 💰 투자 계획 및 수익성 분석
# ================================================================
print("\n💰 투자 계획 및 수익성 분석")
print("-" * 20)

def create_investment_plan(district_type, investment_level):
    """상권 유형별 투자 계획 생성"""
    
    plans = {
        "고투자": {
            "initial_cost": {"min": 80000000, "max": 150000000},
            "monthly_rent": {"min": 800, "max": 1500},
            "interior": {"min": 30000000, "max": 60000000},
            "equipment": {"min": 15000000, "max": 25000000},
            "working_capital": {"min": 10000000, "max": 20000000}
        },
        "중투자": {
            "initial_cost": {"min": 50000000, "max": 80000000},
            "monthly_rent": {"min": 400, "max": 800},
            "interior": {"min": 20000000, "max": 35000000},
            "equipment": {"min": 10000000, "max": 18000000},
            "working_capital": {"min": 8000000, "max": 15000000}
        },
        "저투자": {
            "initial_cost": {"min": 30000000, "max": 50000000},
            "monthly_rent": {"min": 200, "max": 400},
            "interior": {"min": 12000000, "max": 25000000},
            "equipment": {"min": 8000000, "max": 12000000},
            "working_capital": {"min": 5000000, "max": 10000000}
        }
    }
    
    return plans.get(investment_level, plans["중투자"])

# TOP 5 상권 투자 계획
print("💼 TOP 5 상권별 투자 계획:")

for data in strategy_data[:5]:
    plan = create_investment_plan(data['district_type'], data['investment_level'])
    
    print(f"\n🏆 TOP {data['rank']}: {data['name']}")
    print(f"   🏢 상권 유형: {data['district_type']}")
    print(f"   💰 초기 투자: {plan['initial_cost']['min']//10000:,}-{plan['initial_cost']['max']//10000:,}만원")
    print(f"   🏠 월 임대료: {plan['monthly_rent']['min']}-{plan['monthly_rent']['max']}만원")
    print(f"   🎨 인테리어: {plan['interior']['min']//10000:,}-{plan['interior']['max']//10000:,}만원")
    print(f"   🔧 장비비: {plan['equipment']['min']//10000:,}-{plan['equipment']['max']//10000:,}만원")
    print(f"   💵 운전자금: {plan['working_capital']['min']//10000:,}-{plan['working_capital']['max']//10000:,}만원")
    print(f"   📈 예상 ROI: {data['expected_roi']}")
    print(f"   ⏰ 투자회수: {data['payback_period']}")

print("=" * 70)

# 📊 최종 시각화 대시보드
# ================================================================
print("\n📊 최종 종합 대시보드 생성")
print("-" * 20)

# 1. 종합 성과 대시보드
fig_dashboard = make_subplots(
    rows=2, cols=2,
    subplot_titles=('TOP 10 적합도 점수', '상권 유형별 분포', '투자 규모별 분류', '리스크-수익 매트릭스'),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "scatter"}, {"type": "scatter"}]]
)

# TOP 10 적합도 점수
top10_data = top30_df.head(10)
fig_dashboard.add_trace(
    go.Bar(x=top10_data['상권_코드_명'], y=top10_data['곱창집_적합도_v2'],
           name='적합도 점수', marker_color='red'),
    row=1, col=1
)

# 상권 유형별 분포
type_counts = top30_df['상권유형'].value_counts()
fig_dashboard.add_trace(
    go.Pie(labels=type_counts.index, values=type_counts.values, name="상권유형"),
    row=1, col=2
)

# 투자 규모별 분류
investment_data = pd.DataFrame(strategy_data)
colors = {'고투자': 'red', '중투자': 'orange', '저투자': 'green'}
fig_dashboard.add_trace(
    go.Scatter(x=investment_data['score'], y=investment_data['rank'],
               mode='markers+text', text=investment_data['name'],
               marker=dict(color=[colors[inv] for inv in investment_data['investment_level']], size=15),
               name='투자규모별'),
    row=2, col=1
)

# 리스크-수익 매트릭스 (ROI 추정)
roi_numeric = [float(roi.split('-')[0]) for roi in investment_data['expected_roi']]
risk_numeric = [{'저리스크': 1, '중리스크': 2, '고리스크': 3}[risk] for risk in investment_data['risk_level']]

fig_dashboard.add_trace(
    go.Scatter(x=risk_numeric, y=roi_numeric,
               mode='markers+text', text=investment_data['name'],
               marker=dict(size=investment_data['score'], color=investment_data['score'],
                          colorscale='Reds', showscale=True),
               name='리스크-수익'),
    row=2, col=2
)

fig_dashboard.update_layout(
    height=800,
    title_text="🍖 곱창집 입지 분석 종합 대시보드",
    font=dict(family="Arial", size=12)
)

fig_dashboard.show()

# 2. 월별 예상 매출 시뮬레이션 (TOP 3)
fig_revenue = go.Figure()

months = ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']
seasonal_factor = [0.8, 0.7, 0.9, 1.0, 1.1, 1.0, 0.9, 0.9, 1.0, 1.1, 1.2, 1.3]  # 계절성 반영

for i in range(3):
    district = strategy_data[i]
    base_revenue = 50000000 * (district['score'] / 30)  # 기본 월매출 (점수 기반)
    monthly_revenue = [base_revenue * factor for factor in seasonal_factor]
    
    fig_revenue.add_trace(go.Scatter(
        x=months, y=monthly_revenue,
        mode='lines+markers',
        name=f"TOP {i+1}: {district['name'][:15]}",
        line=dict(width=3)
    ))

fig_revenue.update_layout(
    title="📈 TOP 3 상권 월별 예상 매출 시뮬레이션",
    xaxis_title="월",
    yaxis_title="예상 매출 (원)",
    font=dict(family="Arial", size=12),
    height=500
)

fig_revenue.show()

print("=" * 70)

# 📁 최종 보고서 저장
# ================================================================
print("\n📁 최종 보고서 저장")
print("-" * 12)

# Executive Summary 저장
executive_summary = OUTPUT_PATH / "gopchang_executive_summary.txt"
with open(executive_summary, 'w', encoding='utf-8') as f:
    f.write("📋 곱창집 입지 분석 Executive Summary\n")
    f.write(f"📅 보고서 작성일: {datetime.now().strftime('%Y년 %m월 %d일')}\n")
    f.write("=" * 60 + "\n\n")
    
    f.write("🎯 핵심 결론:\n")
    f.write("• 서울시 1,650개 상권 중 TOP 30 곱창집 최적 입지 선정\n")
    f.write(f"• 최우수 상권: {insights['best_overall']['name']} ({insights['best_overall']['score']:.1f}점)\n")
    f.write(f"• 최고 투자효율: {insights['best_value']['name']} (ROI 효율성 최고)\n")
    f.write(f"• 최고 안정성: {insights['most_stable']['name']} (균형잡힌 지표)\n\n")
    
    f.write("📊 시장 규모:\n")
    f.write(f"• TOP 30 상권 야간 유동인구: {total_night_population:,}명\n")
    f.write(f"• 월간 한식 매출: {total_korean_sales/1000000000:,.0f}억원\n")
    f.write(f"• 곱창집 시장 추정: {monthly_market_size:,.0f}억원/월\n\n")
    
    f.write("🏢 상권 유형 분석:\n")
    for idx, row in type_analysis.iterrows():
        f.write(f"• {idx}: {row['상권수']}개 (평균 {row['평균점수']:.1f}점)\n")
    
    f.write("\n💰 투자 추천:\n")
    f.write("• 고투자형: 강남권 (8천만-1.5억원, ROI 20-30%)\n")
    f.write("• 중투자형: 홍대/신촌권 (5천만-8천만원, ROI 15-25%)\n")
    f.write("• 저투자형: 지역거점 (3천만-5천만원, ROI 10-20%)\n")

print(f"✅ Executive Summary: {executive_summary.name}")

# 창업자 실행 가이드 저장
startup_guide = OUTPUT_PATH / "gopchang_startup_guide.txt"
with open(startup_guide, 'w', encoding='utf-8') as f:
    f.write("🍖 곱창집 창업 실행 가이드\n")
    f.write(f"📅 작성일: {datetime.now().strftime('%Y년 %m월 %d일')}\n")
    f.write("=" * 50 + "\n\n")
    
    # TOP 5 상권별 창업 가이드
    f.write("🏆 TOP 5 상권별 창업 전략:\n")
    f.write("=" * 40 + "\n")
    
    for data in strategy_data[:5]:
        plan = create_investment_plan(data['district_type'], data['investment_level'])
        f.write(f"\nTOP {data['rank']}: {data['name']}\n")
        f.write(f"📊 적합도: {data['score']:.1f}점\n")
        f.write(f"🏢 상권유형: {data['district_type']}\n")
        f.write(f"🎯 타겟고객: {data['target']}\n")
        f.write(f"💰 초기투자: {plan['initial_cost']['min']//10000:,}-{plan['initial_cost']['max']//10000:,}만원\n")
        f.write(f"📈 예상ROI: {data['expected_roi']}\n")
        f.write(f"⏰ 투자회수: {data['payback_period']}\n")
        f.write("-" * 40 + "\n")
    
    # 실행 체크리스트
    f.write("\n📋 창업 실행 체크리스트:\n")
    for stage, items in checklist_items.items():
        f.write(f"\n{stage}:\n")
        for i, item in enumerate(items, 1):
            f.write(f"{i}. □ {item}\n")

print(f"✅ 창업 실행 가이드: {startup_guide.name}")

# 투자 계획서 저장
investment_plan_file = OUTPUT_PATH / "gopchang_investment_plan.csv"
investment_df = pd.DataFrame(strategy_data)
investment_df.to_csv(investment_plan_file, index=False, encoding='utf-8-sig')
print(f"✅ 투자 계획서: {investment_plan_file.name}")

print("=" * 70)

# 🎯 프로젝트 완료 요약
# ================================================================
print("\n🎯 곱창집 입지 분석 프로젝트 완료!")
print("-" * 30)

end_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"✅ 프로젝트 완료 시간: {end_time}")

print(f"\n📊 전체 프로젝트 성과:")
print(f"   🔍 분석 상권: {total_districts:,}개 → TOP 30 선정")
print(f"   📈 데이터 기반 KPI: 5개 지표 종합 분석")
print(f"   🗺️ 지도 시각화: 인터랙티브 맵 구현")
print(f"   📋 상권별 전략: 6개 유형별 맞춤 전략")
print(f"   💰 투자 계획: 3단계 투자 규모별 가이드")

print(f"\n🏆 최종 추천 상권 TOP 3:")
for i in range(3):
    data = strategy_data[i]
    print(f"   {i+1}. {data['name']} ({data['score']:.1f}점)")
    print(f"      💰 {data['investment_level']} ({data['initial_cost']})")
    print(f"      📈 ROI: {data['expected_roi']}, 회수: {data['payback_period']}")

print(f"\n📁 생성된 최종 보고서:")
print(f"   📋 gopchang_executive_summary.txt - 경영진용 요약")
print(f"   🍖 gopchang_startup_guide.txt - 창업자용 가이드")
print(f"   💰 gopchang_investment_plan.csv - 투자 계획서")
print(f"   🗺️ gopchang_location_map.html - 인터랙티브 지도")

print(f"\n🚀 다음 단계 권장사항:")
print(f"   1. TOP 3 상권 현장 답사 및 임대료 조사")
print(f"   2. 선택 상권의 경쟁업체 상세 분석")
print(f"   3. 타겟 고객 설문조사 및 메뉴 테스트")
print(f"   4. 사업계획서 작성 및 자금 조달")
print(f"   5. 전문가 컨설팅 및 최종 의사결정")

print("=" * 70)
print("🍖 곱창집 입지 분석 프로젝트 완료!")
print("🎯 데이터 기반 의사결정으로 성공적인 창업을 응원합니다!")